## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-02-23-39-17 +0000,wapo,"Looting, chaos and Israeli gunfire prevent aid...",https://www.washingtonpost.com/world/2025/08/0...
1,2025-08-02-23-34-23 +0000,nyt,Tennessee Man Charged in Attempt to Detonate B...,https://www.nytimes.com/2025/08/02/us/tennesse...
2,2025-08-02-23-18-21 +0000,nypost,Daredevil pooches vie for the title of top dog...,https://nypost.com/2025/08/02/us-news/daredevi...
3,2025-08-02-23-12-02 +0000,bbc,WATCH: How many pull-ups can you do? Trump rev...,https://www.bbc.com/news/videos/ce3j23zk2w2o?a...
4,2025-08-02-23-10-38 +0000,nyt,"Until Trump Fired Her, She Was an Economist Wi...",https://www.nytimes.com/2025/08/02/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,28
178,new,13
227,bar,7
119,city,6
135,texas,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...,74
81,2025-08-02-14-50-09 +0000,bbc,Trump fires lead official on economic data as ...,https://www.bbc.com/news/articles/cvg3xrrzdr0o...,53
153,2025-08-02-02-11-07 +0000,nyt,Stocks Drop as Trump’s New Tariffs Weigh on Ma...,https://www.nytimes.com/2025/08/01/business/ec...,53
121,2025-08-02-09-01-48 +0000,nyt,Will Trump’s Tariffs Close Income Gaps and Bri...,https://www.nytimes.com/2025/08/02/business/ec...,52
75,2025-08-02-15-31-46 +0000,nypost,Trump demurs on pardoning disgraced former Rep...,https://nypost.com/2025/08/02/us-news/presiden...,50


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,74,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...
167,43,2025-08-02-00-59-00 +0000,wsj,The New York Fed’s John Williams says the labo...,https://www.wsj.com/economy/central-banking/fe...
31,36,2025-08-02-20-48-52 +0000,bbc,Police find getaway car of army veteran suspec...,https://www.bbc.com/news/articles/c4gz9ep0wnno...
159,35,2025-08-02-01-28-00 +0000,wsj,"After months of mixed signals, official data s...",https://www.wsj.com/economy/the-wild-week-in-t...
60,30,2025-08-02-18-35-14 +0000,cbc,"U.S. has 'very, very good plan' to end war in ...",https://www.cbc.ca/news/world/us-plan-ceasefir...
46,27,2025-08-02-19-41-50 +0000,nypost,NYPD detective says partner sexually harassed ...,https://nypost.com/2025/08/02/us-news/nypd-cop...
13,26,2025-08-02-22-15-38 +0000,nypost,Theranos fraudster Elizabeth Holmes jogs at ‘C...,https://nypost.com/2025/08/02/us-news/elizabet...
49,26,2025-08-02-19-21-35 +0000,nypost,Three killed in Russia as Ukraine takes aim at...,https://nypost.com/2025/08/02/world-news/three...
174,26,2025-08-02-00-06-30 +0000,nypost,Trans pool player loses court battle in first ...,https://nypost.com/2025/08/01/world-news/trans...
106,25,2025-08-02-10-00-00 +0000,latimes,Indian businesses in SoCal brace for 25% tarif...,https://www.latimes.com/business/story/2025-08...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
